# Use OTEAPI-OPTIMADE with OTElib

[OTElib](https://github.com/EMMC-ASBL/otelib) is a Python client library for the OTEAPI system.
It has two usable backends:

1. Python-based.
   This uses the [OTEAPI Core](https://github.com/EMMC-ASBL/oteapi-core) library directly to exectue pipelines.
   This backend should mainly be used for _development_ purposes.
2. HTTP requests-based.
   This uses the [OTEAPI Services](https://github.com/EMMC-ASBL/oteapi-services) library to execute pipelines via HTTP requests to a running OTEAPI server.
   This backend should be used for _production_ purposes.

This notebook demonstrates how to use OTEAPI-OPTIMADE with OTElib.

## Example

Using OTElib we will do different OPTIMADE queries.
First, we will search through [the Materials Project](https://materialsproject.org) database for all structures with the formula `Al2O3`.
Then, we will search through the Materials Project database for all structures that include `Al` and `O` in their chemical formula.
Finally, we will search through [the Materials Cloud](https://materialscloud.org) [MC3D - Materials Cloud three-dimensional crystals database](https://mc3d.materialscloud.org/) for all structures that include `Al` and `O` in their chemical formula.

## HTTP requests-based backend

This backend is equivalent to running in production.
It requires a running OTEAPI server.

### Setup

First, we need to have a running OTEAPI server.
If one is not available, we can start one using the [container image from GitHub](https://github.com/EMMC-ASBL/oteapi-services/pkgs/container/oteapi).

The documentation for how to run the OTEAPI server is available as [a README on GitHub](https://github.com/EMMC-ASBL/oteapi-services#run-in-docker).
Follow the production target sections.

> **Note**: It is advisable to run the OTEAPI server in a separate terminal window, so that we can see the logs from the server.
> Furthermore, we can stop the server by pressing `Ctrl+C` in the terminal window.

After following the instructions, we should have a running OTEAPI server at [localhost:80](http://localhost:80/docs).

### Create a client

In [1]:
from otelib import OTEClient

client = OTEClient("http://localhost:8080")

#### Search through Materials Project for all structures with the formula `Al2O3`

To search through Materials Project for all structures with the formula `Al2O3`, we need to create the OTE strategies that we want to use for creating the OTE pipeline:

![Pipeline example 1](chem_formula_pipeline.png)

To create the strategies, we need to know how to configure them.
This information is available in the [OTEAPI Core documentation](https://emmc-asbl.github.io/oteapi-core), specifically for [Data Resource strategies](https://emmc-asbl.github.io/oteapi-core/latest/api_reference/models/resourceconfig/#oteapi.models.resourceconfig.ResourceConfig) and for [Filter strategies](https://emmc-asbl.github.io/oteapi-core/latest/api_reference/models/filterconfig/#oteapi.models.filterconfig.FilterConfig).

For the data resource strategy, i.e., the OPTIMADE DB strategy, we need to know the base URL of the OPTIMADE API for Materials Project.
OPTIMADE has a useful [providers dashboard](https://www.optimade.org/providers-dashboard/) that lists all known OPTIMADE providers and their (sub-)databases.
Here we can find the base URL for [the Materials Project](https://www.optimade.org/providers-dashboard/providers/mp.html): `https://optimade.materialsproject.org`.

In [2]:
data_resource_strategy = client.create_dataresource(
    accessService="OPTIMADE",
    accessUrl="https://optimade.materialsproject.org/",
)

For the filter strategy, we need to know the OPTIMADE query that we want to execute.

For retrieving all structures with the formula `Al2O3`, we can use the following OPTIMADE filter query:

```
chemical_formula_descriptive = "Al2O3" OR chemical_formula_reduced = "Al2O3" OR chemical_formula_hill = "Al2O3"
```

In [3]:
filter_strategy = client.create_filter(
    filterType="OPTIMADE",
    query='chemical_formula_descriptive = "Al2O3" OR chemical_formula_reduced = "Al2O3" OR chemical_formula_hill = "Al2O3"',
)

Now we can create the OTE pipeline shown above and execute it.

In [4]:
pipeline = filter_strategy >> data_resource_strategy
session = pipeline.get()

The returned session is a JSON object we can parse and investigate.

In [5]:
import json

parsed_session: dict = json.loads(session)
print(parsed_session.keys())
parsed_session

dict_keys(['optimade_resources', 'optimade_config', 'optimade_resource_model'])


{'optimade_resources': [{'id': 'mp-1228448',
   'type': 'structures',
   'links': None,
   'meta': None,
   'attributes': {'immutable_id': None,
    'last_modified': '2021-03-18T14:42:55+00:00',
    'elements': ['Al', 'O'],
    'nelements': 2,
    'elements_ratios': [0.4, 0.6],
    'chemical_formula_descriptive': 'Al2O3',
    'chemical_formula_reduced': 'Al2O3',
    'chemical_formula_hill': 'Al2O3',
    'chemical_formula_anonymous': 'A3B2',
    'dimension_types': [1, 1, 1],
    'nperiodic_dimensions': 3,
    'lattice_vectors': [[-1.508747, -2.6132, 0.000129],
     [-1.508747, 2.6132, -0.000129],
     [0.0, 0.000184, -7.574636]],
    'cartesian_site_positions': [[-1.508747,
      2.6132805930160004,
      -5.469387346584001],
     [-1.508747, -2.613096593016, -2.105248653416],
     [-1.508747, -0.8710047307360002, -6.063801547042],
     [-1.508747, 0.8711887307360002, -1.510834452958],
     [0.0, 9.2e-05, -3.787318]],
    'nsites': 5,
    'species': [{'name': 'Al',
      'chemical_symbo

As can be seen, there are three keys in the returned session.
`optimade_config` summarizes the query that has been performed to Materials Project.
The OPTIMADE structures are listed under the `optimade_resources`. It is named as such due to there being different OPTIMADE resources, e.g., `structures`, `references`, `links`, etc.
The [OPTIMADE Python tools](https://github.com/Materials-Consortia/optimate-python-tools) has a useful [OPTIMADE Structure model class](http://www.optimade.org/optimade-python-tools/latest/api_reference/adapters/structures/adapter/#optimade.adapters.structures.adapter.Structure) that can be used to parse the OPTIMADE structures into Python objects as well as validating them according to the [OPTIMADE specification](https://github.com/Materials-Consortia/OPTIMADE/blob/master/optimade.rst).
Again, since one can query for different OPTIMADE resources, the specific Python class to use is given in `optimade_resource_model`.

In [6]:
from importlib import import_module

import_path, class_name = parsed_session["optimade_resource_model"].split(":", maxsplit=1)
ResourceClass = getattr(import_module(import_path), class_name)

parsed_structures = [ResourceClass(structure) for structure in parsed_session["optimade_resources"]]
print(f"The query resulted in {len(parsed_structures)} structures found (on page 1) of the returned data.")
print(f"Their Materials Project IDs are: {[structure.id for structure in parsed_structures]}")

The query resulted in 20 structures found (on page 1) of the returned data.
Their Materials Project IDs are: ['mp-1228448', 'mp-755483', 'mp-1245081', 'mp-1244878', 'mp-1105018', 'mp-754401', 'mp-1245063', 'mp-1245265', 'mp-684591', 'mp-1244898', 'mp-1245211', 'mp-1245056', 'mp-642363', 'mp-1244930', 'mp-1244937', 'mp-1244967', 'mp-1244954', 'mp-1244874', 'mp-1245008', 'mp-1245023']


To find them on the Materials Project website, go to `materialsproject.org/materials/<ID>`, for example: [materialsproject.org/materials/mp-1228448](https://materialsproject.org/materials/mp-1228448).

What is more, we can investigate the structure according to the [well-defined OPTIMADE structure model attributes](https://github.com/Materials-Consortia/OPTIMADE/blob/master/optimade.rst#82structures-entries).
For example, so assert the chemical formula is what we expected, we can check the different chemical formula attributes:

In [7]:
structure = parsed_structures[0]
print(structure.id)
for attribute in ("descriptive", "reduced", "hill", "anonymous"):
    print(f"chemical_formula_{attribute}: {getattr(structure, f'chemical_formula_{attribute}', '(not defined)')}")

mp-1228448
chemical_formula_descriptive: Al2O3
chemical_formula_reduced: Al2O3
chemical_formula_hill: Al2O3
chemical_formula_anonymous: A3B2
